In [8]:
import tensorflow as tf
# use keras.datasets instead of tensorflow.datasets
from tensorflow.keras import datasets, layers
mnist = datasets.mnist

learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 5

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = layers.Conv2D(32, 7, activation='relu')
        self.pool1 = layers.MaxPooling2D()
        self.conv2 = layers.Conv2D(64, 5, activation='relu')
        self.pool2 = layers.MaxPooling2D()
        self.flatten = layers.Flatten()
        self.d1 = layers.Dense(1024, activation='relu')
        self.dropout = layers.Dropout(1-keep_prob_rate)
        self.d2 = layers.Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout(x)
        return self.d2(x)

model = MyModel()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

for epoch in range(max_epoch):
    for i, (images, labels) in enumerate(train_ds):
        train_step(images, labels)
        if i % 100 == 0:
            for test_images, test_labels in test_ds:
                test_step(test_images, test_labels)
            template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
            print(template.format(epoch+1,
                                  train_loss.result(),
                                  train_accuracy.result()*100,
                                  test_loss.result(),
                                  test_accuracy.result()*100))

Epoch 1, Loss: 2.315016746520996, Accuracy: 15.625, Test Loss: 2.3037657737731934, Test Accuracy: 9.600000381469727
Epoch 1, Loss: 1.7527433633804321, Accuracy: 54.33168411254883, Test Loss: 1.5834091901779175, Test Accuracy: 45.61499786376953
Epoch 1, Loss: 1.1921391487121582, Accuracy: 68.3302230834961, Test Loss: 1.1949607133865356, Test Accuracy: 60.063331604003906
Epoch 1, Loss: 0.9341874718666077, Accuracy: 74.8338851928711, Test Loss: 0.9798074960708618, Test Accuracy: 67.64500427246094
Epoch 1, Loss: 0.7928770184516907, Accuracy: 78.35879516601562, Test Loss: 0.8417341709136963, Test Accuracy: 72.42599487304688
Epoch 1, Loss: 0.695784866809845, Accuracy: 80.83208465576172, Test Loss: 0.741244375705719, Test Accuracy: 75.89833068847656
Epoch 1, Loss: 0.6265295743942261, Accuracy: 82.73710632324219, Test Loss: 0.6649223566055298, Test Accuracy: 78.4857177734375
Epoch 1, Loss: 0.5710211992263794, Accuracy: 84.2635498046875, Test Loss: 0.6045863032341003, Test Accuracy: 80.49874877